In [24]:
import os

In [25]:
%pwd

'/Users/aminulislam/Desktop/our_value/authentication'

In [3]:
os.chdir("../")

In [26]:
%pwd

'/Users/aminulislam/Desktop/our_value/authentication'

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [34]:
from Authentication.constants import *
from Authentication.utils.common import read_yaml, create_directories

In [35]:
class ConfigurationManager:
    
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
        

In [36]:
import os
import urllib.request as request
import zipfile
from Authentication import logger
from Authentication.utils.common import get_size

In [37]:
class DataIngestion:
    
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exits of size: {get_size(Path(self.config.local_data_file))}")
            
            
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
 
        """
        unzip_path = self.config.unzip_dir
        
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
            

In [38]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2023-09-05 04:30:29,419: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-05 04:30:29,423: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-05 04:30:29,425: INFO: common: created directory at: artifacts]
[2023-09-05 04:30:29,427: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-05 04:31:22,754: INFO: 3120940718: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E51A:1783:5ADD4:71B55:64F65A86
Accept-Ranges: bytes
Date: Mon, 04 Sep 2023 22:31:19 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1269-QPG
X-Cache: MISS